In [1]:
import os, csv
from dotenv import load_dotenv
import time
from datetime import datetime
import schedule
import requests

import pandas as pd
import pandas_ta as ta
import numpy as np

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
load_dotenv()

True

In [2]:
API_KEYS = os.getenv('ALPHA_KEYS')
cc = ForeignExchange(key=API_KEYS)

LINE_TOKEN = os.getenv('LINE_TOKEN')
LINE_URL = 'https://notify-api.line.me/api/notify'
LINE_HEADERS = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + LINE_TOKEN }

ts = TimeSeries(key=API_KEYS, output_format='pandas')

In [3]:
class SimpleBbandsNotify:
    def __init__(self, params, ts):
        self.ts = ts
        self.pair = params['pair']
        self.tf = params['tf']
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.bbands = params['bbands']
        self.rsi = params['rsi']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.bars_status = 'idle' # idle, break_high, break_low, overbought, oversold
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        
        self.line_url = params['line_url']
        self.line_token = params['line_token']
        self.line_headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer ' + self.line_token }
        self.df = None
        
    def notifyMsg(self, *msg):
        r = requests.post(self.line_url, headers=self.line_headers, data={'message': msg[0]})
        print(r.text)
        
    def initial_bars(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='full')
        df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with strategy

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.iloc[::-1] # reverse row
        df.reset_index(drop=True, inplace=True)

        self.df = df
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
    
    def check_breakout(self, row):
        if row['close'] > row['open']:
            
            if (row['BBU_' + str(self.bbands) + '_2.0'] < row['close']) and \
            row['BBU_' + str(self.bbands) + '_2.0'] > row['open'] and \
            self.bars_status == 'oversold':
                self.bars_status = 'break_high'
                
            elif (row['BBL_' + str(self.bbands) + '_2.0'] < row['close']) and \
            row['BBL_' + str(self.bbands) + '_2.0'] > row['open'] and \
            self.bars_status == 'overbought':
                self.bars_status = 'break_low'
                
        elif row['close'] < row['open']:
            
            if (row['BBU_' + str(self.bbands) + '_2.0'] > row['close']) and \
            row['BBU_' + str(self.bbands) + '_2.0'] < row['open'] and \
            self.bars_status == 'oversold':
                self.bars_status = 'break_high'
                
            elif (row['BBL_' + str(self.bbands) + '_2.0'] > row['close']) and \
            row['BBL_' + str(self.bbands) + '_2.0'] < row['open'] and \
            self.bars_status == 'overbought':
                self.bars_status = 'break_low'
                
        else:
            self.bars_status = 'idle'
        
    def check_over_rsi(self, row):
        if row['RSI_' + str(self.rsi)] > self.overbought:
                self.bars_status = 'overbought'
        elif row['RSI_' + str(self.rsi)] < self.oversold:
                self.bars_status = 'oversold'
                
    def check_doji(self, row, index):
        if self.bars_status == 'break_low': # buy
            if self.df.iloc[index-1]['close'] < self.df.iloc[index-1]['open']:
                if ((row['close'] > row['open']) or (round(row['close'], 4) == round(row['open'], 4))):
                    return 'buy'
        elif self.bars_status == 'break_high': # sell
            if self.df.iloc[index-1]['close'] > self.df.iloc[index-1]['open']:
                if ((row['close'] < row['open']) or (round(row['close'], 4) == round(row['open'], 4))):
                    return 'sell'
        return None
    
    def get_realtime(self):
        bars, _ = self.ts.get_intraday(symbol=self.pair,interval=self.tf, outputsize='compact')
        incoming_df = pd.DataFrame({
            'timestamp': bars.index,
            'open': bars['1. open'],
            'high': bars['2. high'],
            'low': bars['3. low'],
            'close': bars['4. close'],
            'volumn': bars['5. volume']
        }) # convert alpha vantage data to use with rsi strategy

        incoming_df['timestamp'] = pd.to_datetime(incoming_df['timestamp'], unit='ms')
     
        if (incoming_df['timestamp'].iloc[0] != self.df['timestamp'].iloc[-1]):
            self.df = self.df[:900].append(incoming_df.head(1), ignore_index = True)

        bollinger_rsi = ta.Strategy(
            name = 'Bollinger Band and RSI',
            description = 'Bollinger Band and RSI strategy',
            ta = [
                {'kind': 'bbands', 'length': self.bbands},
                {'kind': 'rsi', 'length': self.rsi},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        
        self.df.ta.strategy(bollinger_rsi)
        row = self.df.tail(1).to_dict('records')[0]
        
        if self.current_action == 'close' or self.order == {}:
            if self.bars_status == 'idle':
                self.check_over_rsi(row)
                    
            if (self.bars_status == 'overbought') or (self.bars_status == 'oversold'):
                self.check_breakout(row)
                    
            elif (self.bars_status == 'break_high') or (self.bars_status == 'break_low'):
                action = self.check_doji(row, index)
                
                if action == 'buy':
                    self.bars_status = 'idle'
                    row['action'] = 'buy'
                    self.order['action'] = 'buy'
                    self.order['open time'] = row['timestamp']
                        
                    self.order['open'] = row['close']
                    self.order['S/L'] = row['close'] - row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['close'] + (self.rr * abs(row['close'] - self.order['S/L']))
                    self.current_action = 'buy'
                    
                elif action == 'sell':
                    self.bars_status = 'idle'
                    row['action'] = 'sell'
                    self.order['action'] = 'sell'
                    self.order['open time'] = row['timestamp']
                        
                    self.order['open'] = row['close']
                    self.order['S/L'] = row['close'] + row['ATRr_'+str(self.atr)]
                    self.order['T/P'] = row['close'] - (self.rr * abs(row['close'] - self.order['S/L']))
                    self.current_action = 'sell'
                        
                else:
                    self.current_action == 'close'
                    self.bars_status == 'idle'

        if (self.current_action == 'buy'):
            if (row['low'] <= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.notifyMsg(f"S/L Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()
                
            elif (row['high'] >= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.notifyMsg(f"T/P Buy {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

        elif (self.current_action == 'sell'):
            if (row['high'] >= self.order['S/L']):
                self.order['result'] = 'S/L'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['S/L']
                
                self.current_action = 'close'
                self.notifyMsg(f"S/L Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()
                
            elif (row['low'] <= self.order['T/P']):
                self.order['result'] = 'T/P'
                self.order['close time'] = row['timestamp']
                self.order['close'] = self.order['T/P']
                
                self.current_action = 'close'
                self.notifyMsg(f"T/P Sell {self.pair} RSI_O2\nprice: {self.order['close']}")
                self.reset_order()

In [6]:
usdjpy = SimpleBbandsNotify({
    'pair': 'USDJPY',
    'tf': '15min',
    'atr': 14,
    'bbands': 30,
    'rsi': 14,
    'pip_value': 0.001,
    'rr': 1.5,
    'overbought': 70,
    'oversold': 30,
    'line_url': LINE_URL,
    'line_token': LINE_TOKEN,
}, ts)

In [7]:
usdjpy.initial_bars()

In [8]:
def get_all_realtime():
    usdjpy.get_realtime()
    
schedule.every(60 * 15).seconds.do(get_all_realtime) 

Every 900 seconds do get_all_realtime() (last run: [never], next run: 2022-03-13 11:44:02)

In [9]:
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 